# Social auth

In [2]:
import spotipy
from datetime import datetime, timedelta
from social_django.utils import load_strategy

In [20]:
def get_spotipy(social):
    expired_time = datetime.fromtimestamp(social.extra_data['auth_time']) + timedelta(minutes=30)
    if expired_time < datetime.now():
        social.refresh_token(load_strategy())

    access_token = social.get_access_token(load_strategy())
    sp = spotipy.Spotify(auth=access_token)
    return sp

In [25]:
def insert_tracks(tracks, user):
    for track_dic in tracks[::-1]:
        track = Track(
            track_id=track_dic['track']['id'],
            name=track_dic['track']['name'], 
            artist_id=track_dic['track']['artists'][0]['id'],
            artist_name=track_dic['track']['artists'][0]['name'],
            album_id=track_dic['track']['album']['id'],
            album_name=track_dic['track']['album']['name'],
            popularity=track_dic['track']['popularity'],
            url=track_dic['track']['href'],
            preview_url=track_dic['track']['preview_url'],
            release_date=track_dic['track']['album']['release_date'],
            played_at=track_dic['played_at'],
            user=user
        )
        track.save()

In [26]:
for user in User.objects.all():
    if user.social_auth.exists():
        social = user.social_auth.get(provider='spotify')
        sp = get_spotipy(social)
        if Track.objects.filter(user=user).exists():
            latest = Track.objects.filter(user=user).order_by('-played_at')[0]
            tracks = sp.current_user_recently_played(after=int(latest.played_at.timestamp() * 1000))['items']
        else:
            tracks = sp.current_user_recently_played()['items']
        insert_tracks(tracks, user)

In [27]:
# me = User.objects.get(username="21bpmv7tcs4av5sakwfxkof2a")
# social = me.social_auth.get(provider='spotify')

In [32]:
access_token = social.get_access_token(load_strategy())
sp = spotipy.Spotify(auth=access_token)
sp.current_user()

{'country': 'JP',
 'display_name': 'Yutaro Ikeda',
 'email': 'ikeda.yutaro@gmail.com',
 'explicit_content': {'filter_enabled': False, 'filter_locked': False},
 'external_urls': {'spotify': 'https://open.spotify.com/user/21bpmv7tcs4av5sakwfxkof2a'},
 'followers': {'href': None, 'total': 9},
 'href': 'https://api.spotify.com/v1/users/21bpmv7tcs4av5sakwfxkof2a',
 'id': '21bpmv7tcs4av5sakwfxkof2a',
 'images': [{'height': None,
   'url': 'https://platform-lookaside.fbsbx.com/platform/profilepic/?asid=629148017161705&height=300&width=300&ext=1593986408&hash=AeQYY0sX7Q_CTlvQ',
   'width': None}],
 'product': 'premium',
 'type': 'user',
 'uri': 'spotify:user:21bpmv7tcs4av5sakwfxkof2a'}

In [7]:
sp.currently_playing()

In [8]:
tracks = sp.current_user_recently_played()

In [9]:
tracks['items'][0].keys()

dict_keys(['track', 'played_at', 'context'])

In [10]:
tracks['items'][0]['played_at']

'2020-06-04T11:28:50.645Z'

In [11]:
tracks['items'][0]['context']

{'external_urls': {'spotify': 'https://open.spotify.com/artist/4AGNJdJiVltImYk1UTLE0K'},
 'href': 'https://api.spotify.com/v1/artists/4AGNJdJiVltImYk1UTLE0K',
 'type': 'artist',
 'uri': 'spotify:artist:4AGNJdJiVltImYk1UTLE0K'}

In [12]:
track = tracks['items'][0]['track']
track.keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [13]:
track['album'].keys()

dict_keys(['album_type', 'artists', 'available_markets', 'external_urls', 'href', 'id', 'images', 'name', 'release_date', 'release_date_precision', 'total_tracks', 'type', 'uri'])

In [14]:
track['album']['images'][0]['url']

'https://i.scdn.co/image/ab67616d0000b273acd8242f2e56f9b7be5d4246'

In [15]:
[track['track']['name'] for track in tracks['items']][:10]

['Dina',
 'Feels Like Magic',
 'You Are the Right One',
 "People Can't Stop Chillin'",
 'Whatever You Want',
 "Gotta Know Better (I'm Crazy)",
 "Takin' a Cruise",
 "Drivin' on by You",
 'Manicure',
 "Someone You'd Rather Be Dating"]

In [16]:
#     name =  models.TextField(max_length=128)
#     artist_name =  models.TextField(max_length=128)
#     album_name =  models.TextField(max_length=128)
#     popularity = models.IntegerField()
#     url = models.URLField()
#     preview_url = models.URLField()
#     release_date = models.DateField()
#     played_at = models.DateTimeField()
#     added_at = models.DateTimeField(default=timezone.now)
#     user = models.ForeignKey(User, on_delete=models.CASCADE)

In [17]:
for track_dic in tracks['items']:
    break

In [18]:
track_dic['track']['artists'][0]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/4AGNJdJiVltImYk1UTLE0K'},
 'href': 'https://api.spotify.com/v1/artists/4AGNJdJiVltImYk1UTLE0K',
 'id': '4AGNJdJiVltImYk1UTLE0K',
 'name': 'Sports',
 'type': 'artist',
 'uri': 'spotify:artist:4AGNJdJiVltImYk1UTLE0K'}

In [19]:
track_dic['track']['album']['id']

'5mzRPtsvAzLBqyYtjH6gMs'

In [20]:
track_dic['track']['album']['name']

'Naked All the Time'

In [21]:
track_dic['track']['name']

'Dina'

In [22]:
track_dic['track']['popularity']

34

In [23]:
track_dic['track']['href']

'https://api.spotify.com/v1/tracks/6h8902K3bu35UpmS0x5GBU'

In [24]:
track = Track(
    track_id=track_dic['track']['id'],
    name=track_dic['track']['name'], 
    aartist_id=track_dic['track']['artists'][0]['id'],
    artist_name=track_dic['track']['artists'][0]['name'],
    album_id=track_dic['track']['album']['id'],
    album_name=track_dic['track']['album']['name'],
    popularity=track_dic['track']['popularity'],
    url=track_dic['track']['href'],
    preview_url=track_dic['track']['preview_url'],
    release_date=track_dic['track']['album']['release_date'],
    played_at=track_dic['played_at'],
    user=me
)

In [26]:
track.save()

In [28]:
track_dic['played_at']

'2020-06-04T11:28:50.645Z'

In [76]:
def insert_tracks(tracks):
    for track_dic in tracks[::-1]:
        track = Track(
            track_id=track_dic['track']['id'],
            name=track_dic['track']['name'], 
            aartist_id=track_dic['track']['artists'][0]['id'],
            artist_name=track_dic['track']['artists'][0]['name'],
            album_id=track_dic['track']['album']['id'],
            album_name=track_dic['track']['album']['name'],
            popularity=track_dic['track']['popularity'],
            url=track_dic['track']['href'],
            preview_url=track_dic['track']['preview_url'],
            release_date=track_dic['track']['album']['release_date'],
            played_at=track_dic['played_at'],
            user=me
        )
        track.save()

In [95]:
if Track.objects.filter(user=me).exists():
    latest = Track.objects.filter(user=me).order_by('-played_at')[0]
    tracks = sp.current_user_recently_played(after=int(latest.played_at.timestamp() * 1000))['items']
else:
    tracks = sp.current_user_recently_played()['items']
insert_tracks(tracks)

0
